In [1]:
from tensorflow_addons.metrics import RSquare
from keras.models import load_model
import pandas as pd
import numpy as np
import joblib

2024-04-30 15:44:08.911774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 15:44:09.027643: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-30 15:44:09.520468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/baran-tomik/bin/miniconda3/envs/tf/lib/
2024-04-30 15:44:09.520565: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvin

### Mean Average Precision - MAP@k

In [2]:
# modified from:
# author: Ben Hamner
# author's github: benhamner
# link to github: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py 

def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    apk_sum = 0.0
    for user in actual:
        if user in predicted:
            apk_sum += apk(actual[user], predicted[user], k)

    return apk_sum / len(actual)

### Mean Reciprocal Rank - MRR

In [3]:
def mrr(actual, predicted):
    mrr_sum = 0.0
    for user in actual:
        if user in predicted:
            rank = 1
            for movie in predicted[user]:
                if movie in actual[user]:
                    mrr_sum += 1.0 / rank
                    break
                rank += 1
    return mrr_sum / len(actual)

-----------

In [4]:
# ## 25m Added IMDb context
# test_split_data_path = 'eval_data/test_split_25ml_added_imdb_context.csv'
# recsys_data_path = '../data/transform_data/25m/'
# nn_model_path = '../model/arch5_25m_added_imdb_context_max_abs_scaler_gc_trained.keras'
# scaler_file = '25m_added_imdb_context_scaler.pkl'

# 25m MovieLens context - label encode just 'holiday'
# test_split_data_path = 'eval_data/test_split_25ml_movielens_context.csv'
# recsys_data_path = '../data/transform_data/25m_movielens_context/'
# nn_model_path = '../model/arch5_25m_movielens_context_max_abs_scaler_gc_2_trained.keras'
# scaler_file = '25m_movielens_context_scaler.pkl'

# ## 25m No context - do not label encode any data
# test_split_data_path = 'eval_data/test_split_25ml_no_context.csv'
# recsys_data_path = '../data/transform_data/25m_no_context/'
# nn_model_path = '../model/arch5_25m_no_context_max_abs_scaler_gc_trained.keras'
# scaler_file = '25m_no_context_scaler.pkl'

# # ## 1m No context - do not label encode any data
# test_split_data_path = 'eval_data/test_split_1ml_no_context.csv'
# recsys_data_path = '../data/transform_data/1m_no_context/'
# nn_model_path = '../model/arch5_1m_no_context_max_abs_scaler_gc_trained.keras'
# scaler_file = '1m_no_context_scaler.pkl'

# # # ## 1m MovieLens context - label encode just 'holiday' and 'user_gender'
# test_split_data_path = 'eval_data/test_split_1ml_movielens_context.csv'
# recsys_data_path = '../data/transform_data/1m_movielens_context/'
# nn_model_path = '../model/arch5_1m_movielens_context_max_abs_scaler_gc_trained.keras'
# scaler_file = '1m_movielens_context_scaler.pkl'

# # ## 1m Added IMDb context
# test_split_data_path = 'eval_data/test_split_1ml_added_imdb_context.csv'
# recsys_data_path = '../data/transform_data/1m/'
# nn_model_path = '../model/arch5_1m_added_imdb_context_max_abs_scaler_gc_trained.keras'
# scaler_file = '1m_added_imdb_context_scaler.pkl'

In [5]:
test_ratings = pd.read_csv(test_split_data_path)
test_ratings

,user_id,movie_id,rating
0,2857,2268,4
1,1172,2710,3
2,5376,3681,5
3,4362,246,5
4,2197,246,4
...,...,...,...
100016,4253,1348,2
100017,3367,2080,4
100018,261,3698,1
100019,2777,1665,2


In [7]:
# number of users to evaluate
n_users = 10
# number of recommendations to make
top_k = 10
# lowest rating to consider as a positive recommendation
low_rating = 4.0

# sort users by number of ratings
test_ratings_all_users = test_ratings['user_id'].value_counts().index.tolist()[:n_users]

nn_model = load_model(nn_model_path, custom_objects={'RSquare': RSquare()}, compile=True)

2024-04-30 15:44:10.351647: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-30 15:44:10.373973: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-30 15:44:10.374142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-30 15:44:10.374660: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
def scale_data(data):
    print(data.shape)
    # actor_label_encoder = joblib.load(recsys_data_path + 'actor_label_encoder.pkl')
    # directors_label_encoder = joblib.load(recsys_data_path + 'directors_label_encoder.pkl')
    # holiday_label_encoder = joblib.load(recsys_data_path + 'holiday_label_encoder.pkl')
    # titleType_label_encoder = joblib.load(recsys_data_path + 'titleType_label_encoder.pkl')
    # user_gender_label_encoder = joblib.load(recsys_data_path + 'user_gender_label_encoder.pkl')

    # Load scaler
    scaler = joblib.load(recsys_data_path + scaler_file)

    # Label encode data
    # data['actor'] = actor_label_encoder.transform(data['actor'])
    # data['directors'] = directors_label_encoder.transform(data['directors'])
    # data['holiday'] = holiday_label_encoder.transform(data['holiday'])
    # data['titleType'] = titleType_label_encoder.transform(data['titleType'])
    # data['user_gender'] = user_gender_label_encoder.transform(data['user_gender'])


    # Return scaled data
    return scaler.transform(data)

In [9]:
test_ratings_all_users  # 1m

[4169, 1680, 1941, 1181, 4277, 889, 5795, 2063, 2909, 3391]

In [10]:
actual = {}
predicted = {}

for user_id in test_ratings_all_users:

    user_test_ratings = test_ratings[test_ratings['user_id'] == user_id]

    actual_ratings = user_test_ratings[user_test_ratings['rating'] >= low_rating]
    actual[user_id] = actual_ratings['movie_id'].tolist()

    scaled_user_test_ratings = scale_data(user_test_ratings.drop(['rating'], axis=1))
    movie_indices = user_test_ratings['movie_id'].values

    predictions = nn_model.predict(scaled_user_test_ratings, verbose=0).flatten()

    predicted_movies = pd.DataFrame({'movie_id': movie_indices, 'rating': predictions})
    predicted_movies = predicted_movies.sort_values(by='rating', ascending=False)
    predicted[user_id] = predicted_movies['movie_id'][:top_k].tolist()

(225, 2)


/home/baran-tomik/bin/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MaxAbsScaler from version 1.2.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/baran-tomik/bin/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MaxAbsScaler from version 1.2.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/baran-tomik/bin/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MaxAbsScaler 

(171, 2)
(164, 2)
(162, 2)
(153, 2)
(150, 2)


/home/baran-tomik/bin/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MaxAbsScaler from version 1.2.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/baran-tomik/bin/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MaxAbsScaler from version 1.2.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/baran-tomik/bin/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MaxAbsScaler 

(148, 2)
(137, 2)
(135, 2)
(133, 2)


/home/baran-tomik/bin/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MaxAbsScaler from version 1.2.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/baran-tomik/bin/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MaxAbsScaler from version 1.2.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
# profile 1 - 72315 | profile 2 - 80974 | profile 3 - 107650

----

In [16]:
# MRR
mrr_fin = mrr(actual, predicted)
print('MRR: ', mrr_fin) 

TypeError: 'float' object is not callable

In [20]:
# MAP@K
mapa3 = mapk(actual, predicted, 3)
mapa10 = mapk(actual, predicted, 10)
print('MAP@3:  ', mapa3)
print('MAP@10: ', mapa10)

MAP@3:   0.5166666666666666
MAP@10:  0.4833373015873015
